In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

def ReadData(X_file, y_file):
    X = np.loadtxt(X_file)
    y = np.loadtxt(y_file)
    yhat = [2*float(i)-1. for i in y]
    return X, yhat, X.shape[0], X.shape[1]

X, y, m, n = ReadData('q1x.txt', 'q1y.txt')

In [ ]:
w = cp.Variable(n)
b = cp.Variable(1)
xi = cp.Variable(m)

In [ ]:
def PlotData(w,b):
    xp = np.linspace(np.min(X[:,0]), np.max(X[:,0]), 100)
    yp = - (w[0].value*xp + b.value)/w[1].value
    yp1 = - (w[0].value*xp + b.value - 1)/w[1].value
    yp0 = - (w[0].value*xp + b.value + 1)/w[1].value
    
    XX, yy = pd.DataFrame(X), pd.DataFrame(y)
    XX1, XX2 = XX[yy[0]==1], XX[yy[0]==-1]
    
    plt.scatter(XX1[0],XX1[1],color='blue')
    plt.scatter(XX2[0],XX2[1],color='red')
    plt.plot(xp,yp,'k-',xp,yp1,'m--',xp,yp0,'g--')
    plt.show()

### Soft Margin SVM

minimize 
> $\frac { 1 } { 2 } \| \mathbf { w } \| _ { 2 } ^ { 2 } + C \sum _ { i = 1 } ^ { N } \xi _ { i }$

subject to 
> $y _ { i } \left( \mathbf { w } ^ { T } \mathbf { x } _ { i } + b \right) \geq 1 - \xi _ { i } , i = 1 , \cdots , N$ and $\xi _ { i } \geq 0$

In [ ]:
C = 1.

objective = cp.Minimize(0.5*cp.sum_squares(w)+C*cp.sum_entries(xi))
constraints = [y[i]*((X*w)[i]+b) >= 1-xi[i] for i in range(m)]
constraints.append(xi >= 0)

prob = cp.Problem(objective, constraints)

try:
    result = prob.solve()
    PlotData(w,b)
except Exception as exc:
    print('{}'.format(exc))

In [ ]:
def SubPlotData(ax, w, b):
    ax.set_title('C = ' + str(C))
    xp = np.linspace(np.min(X[:,0]), np.max(X[:,0]), 100)
    yp = - (w[0].value*xp + b.value)/w[1].value
    yp1 = - (w[0].value*xp + b.value - 1)/w[1].value
    yp0 = - (w[0].value*xp + b.value + 1)/w[1].value
    
    XX, yy = pd.DataFrame(X), pd.DataFrame(y)
    XX1, XX2 = XX[yy[0]==1], XX[yy[0]==-1]
    
    plt.scatter(XX1[0],XX1[1],color='blue')
    plt.scatter(XX2[0],XX2[1],color='red')
    plt.plot(xp,yp,'k-',xp,yp1,'m--',xp,yp0,'g--')

### Try different penalty parameters C

In [ ]:
Clist = [0, 0.001, 0.01, 1, 1000000]

with mpl.rc_context(rc={'font.family': 'serif', 'font.size': 11}):
    fig = plt.figure(figsize=(16,3.5))
    for i in range(len(Clist)):
        C = Clist[i]
        objective = cp.Minimize(0.5*cp.sum_squares(w)+C*cp.sum_entries(xi))
        prob = cp.Problem(objective, constraints)
        try:
            result = prob.solve()
            ax = fig.add_subplot(int('14'+str(i+1)))
            SubPlotData(ax,w,b)
        except Exception as exc:
            print('Error: C = ' + str(C) + ': {}'.format(exc))